# Uber Prize Starter Kit Python Utilities: Tutorial

To simplify interaction with the Docker-based simulation execution and evaluation, we've provided a set of Python utilities (located in the `/utilities` folder of the Starter-Kit repository).This notebook demonstrates how they may be used to accomplish the following tasks:

 - Starting a simulation or several simulations
 - Checking simulation completion
 - Retrieving the simulation score in a convenient Pandas `DataFrame` format.
 - Generating fake data 

*Note 1*: This notebook and accompanying utilities are written in Python 3.5+. Please install associated requirements using the provided `requirements.txt` in the root folder.

*Note 2*: It is assumed that this notebook is started from the `/examples` folder.

In [1]:
# Adding the module to the path for future import
import sys
import os
import docker
from pathlib import Path
# Note that the following is idempotent when this notebook is run from "/examples"
os.chdir('../utilities')
%load_ext autoreload
%autoreload 2

## Running a simulation: the `competition_executor` module

A `CompetitionContainerExecutor` object may be used to start, stop, and gather information about containers running simulations and/or completed simulation scores and stats.

In [10]:
from competition_executor import CompetitionContainerExecutor

# Current round uses "sioux faux" scenario
SCENARIO_NAME = "sioux_faux"

# Evaluation is on 15k sample, but for now we will use the 1k sample for demo purposes.
SAMPLE_SIZE = "1k"

path_input = (Path(Path.cwd()).parent / "submission-inputs").absolute()
path_output = (Path(Path.cwd()).parent / "output").absolute()

my_executor = CompetitionContainerExecutor(input_root=path_input,
                                  output_root=path_output)

# Note: Instantiating a CompetitionContainerExecutor with path arguments is not strictly necessary. Each simulation
# can be run with its own set of input and/or output arguments. However, if you prefer to designate
# a single directory for inputs and/or a single directory for outputs, then, for convenience, 
# you may pass those arguments in here and avoid re-entering them for each simulation run.

Run the simulation using the `run_sumulation` method. For example:

In [11]:
# Note: the `submission name` for each run must be unique. However, this cell is designed to be run repeatedly. 
# If a simulation is currently in progress, it will be stopped and removed.
try:
    my_submission = my_executor.client.containers.get('my_submission')
    print("WARNING: The executor found a simulation named '{0}' (container ID '{1}') from a previous run.\n\
    - Stopping simulation and removing the old container.\n \
    - Creating new container and restarting simulator.\n\n \
          ~~~Please wait a moment~~~".format(my_submission.name, my_submission.short_id))
    my_submission.stop()
    my_submission.remove()
except docker.errors.NotFound:
    print("Creating new container and starting simulator.\n \
    ~~~Please wait a moment~~~\n")
    
my_executor.run_simulation('my_submission', num_iterations=1, num_cpus=2)

# Note that a dictionary indicating which simulation names already exist can be accessed.
for key, value in my_executor.containers.items():
    
    print("\nSuccessfully created new container and currently executing a simulation run\n \
    - Name: '{0}'\n \
    - Container ID : '{1}'\n \
    \n \
    ~~~Scores will be available when this run completes.~~~\n".format(value.name, value.id))

Creating new container and starting simulator.
     ~~~Please wait a moment~~~


Successfully created new container and currently executing a simulation run
     - Name: 'my_submission'
     - Container ID : '0aef3693bc'
     
     ~~~Scores will be available when this run completes.~~~



You may view (or save to file) the **debug logs** of your simulation using the `output_simulation_logs(simulation_name,file_name)` method on the executor. 

To follow simulation progress, the following cell may be run repeatedly. 

In [34]:
logs = my_executor.output_simulation_logs('my_submission')

# NOTE: While it may take a while to finish, as long as your execution environment meets 
# the minimum hardware requirements the  simulation is (most likely) not stuck! 

# However, if you notice that the logs do not change for 
# over 5 minutes, there is likely an error somewhere and you should stop the simulation (see below). 
# In this case, we would greatly appreciate it if you filed a bug report (issue) 
# together with your hardware environment (number of cpus, RAM, etc.) and a copy of the output 
# logs (zipped, preferably).

Dependency Service(s) Init Complete
7:C 12 Feb 22:05:10.531 # Warning: no config file specified, using the default config. In order to specify a config file use redis-server /path/to/redis.conf
                _._                                                  
           _.-``__ ''-._                                             
      _.-``    `.  `_.  ''-._           Redis 3.2.6 (00000000/0) 64 bit
  .-`` .-```.  ```\/    _.,_ ''-._                                   
 (    '      ,       .-`  | `,    )     Running in standalone mode
 |`-._`-...-` __...-.``-._|'` _.-'|     Port: 6379
 |    `-._   `._    /     _.-'    |     PID: 7
  `-._    `-._  `-./  _.-'    _.-'                                   
 |`-._`-._    `-.__.-'    _.-'_.-'|                                  
 |    `-._`-._        _.-'_.-'    |           http://redis.io        
  `-._    `-._`-.__.-'_.-'    _.-'                                   
 |`-._`-._    `-.__.-'    _.-'_.-'|                                  
 |    `-.

Submissions run for a certain amount of time. If you interrupt it before the end, you will not get any outputs. You can **check if a submission is finished** with the following method:

In [35]:
my_executor.check_if_submission_complete('my_submission')

True

When a simulation run is done, you can import its **results**:

In [36]:
scores, stats = my_executor.get_submission_scores_and_stats('my_submission')

In [37]:
scores['Raw Score']

Component Name
Level of service: costs and benefits                                         1.047682
Accessibility: Number of secondary locations accessible within 15 minutes    1.111695
Accessibility: Number of work locations accessible within 15 minutes         1.073466
Sustainability: Total PM 2.5 Emissions                                       0.999509
Level of service: average trip expenditure - work                            1.104799
Congestion: average vehicle delay per passenger trip                         1.069325
Level of service: average bus crowding experienced                           0.978100
Level of service: average on-demand ride wait times                          0.957241
Congestion: total vehicle miles traveled                                     1.005112
Level of service: average trip expenditure - secondary                       1.060200
Submission Score                                                                  NaN
Name: Raw Score, dtype: float64

The **scores** and **statistics** are stored in pandas DataFrames which contains the information described [here](https://github.com/vgolfier/Uber-Prize-Starter-Kit/blob/master/docs/Understanding_the_outputs_and_the%20scoring_function.md).

At the end of any run and to avoid any conflicts between submission ids (particularly if you want to reuse names), it is advised to **clean up containers** calling the following method:

In [39]:
my_executor.stop_all_simulations()

## The `input_sampler` module:

Randomly sampled data may be used to either initialize the re-planning algorithm or otherwise test the simulation. The `input_sampler` has been provided towards this end. This subsection provides an example of how synthetic random input files may be generated for each of the available input policies to the simulation.

In [40]:
from input_sampler import *
# Specify the common string for the scenario name 
# (currently only "siouxfalls", which refers to the Sioux Falls scenario).
agency = 'sioux_faux_bus_lines'

# Set the paths appropriately
DIR = Path(__name__).absolute().parent.parent
sys.path.append(str(DIR))
DATA_DIR = DIR / 'reference-data/'

agency_dict = scenario_agencies(DATA_DIR,SCENARIO_NAME)
    # Create a lazy cache of GTFS data for the agency:
sf_gtfs_manager = AgencyGtfsDataManager(agency_dict[agency])

# Sample each input five (5) times (naming of functions is indicative of type of input sampled).
freq_df = sample_frequency_adjustment_input(5, sf_gtfs_manager)
mode_incentive_df = sample_mode_incentives_input(5)
vehicle_fleet_mix_df = sample_vehicle_fleet_mix_input(5, sf_gtfs_manager)

# Generated inputs may now be saved in, say, /submission-inputs or some other directory. 
# Remember the location of this directory when executing new simulations.

In [41]:
vehicle_fleet_mix_df

,agencyId,routeId,vehicleTypeId
0,217,1346,BUS-SMALL-HD
1,217,1351,BUS-SMALL-HD
2,217,1340,BUS-SMALL-HD
3,217,1345,BUS-STD-ART
4,217,1349,BUS-DEFAULT


In [42]:
mode_incentive_df

,mode,age,income,amount
0,drive_transit,(54:80),(116000:232000),15.3
1,ondemand_ride,(34:47),[96000:226000),16.1
2,ondemand_ride,(15:97],(7000:270000],8.8
3,ondemand_ride,(92:96],[191000:272000],19.2
4,ondemand_ride,[29:98],(58000:169000),16.3


In [43]:
freq_df

,trip_id,start_time,end_time,headway_secs,exact_times
0,t_500923_b_219_tn_0,19140,45660,3420,0
1,t_500992_b_219_tn_0,20700,56100,5520,0
2,t_75320_b_219_tn_3,40260,70200,4020,0
3,t_72465_b_219_tn_21,4260,12480,2460,0
4,t_72462_b_219_tn_11,7020,65460,6900,0
